<a href="https://colab.research.google.com/github/PaulinaRuizB/Machine_Learning_Theory/blob/main/Parcial1/4_DataSet_NFL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#PARCIAL 1 DE TEORÍA DE APRENDIZAJE DE MÁQUINA 2025-2
##Paulina Ruiz bonilla
##Juan David Lopez Rios



---



#Punto #4.1: Análisis
Presente un analisis detallado del conjunto de datos disponible en https://www.kaggle.com/competitions/nfl-big-data-bowl-2026-prediction/overview. Incluya la descripcion de las variables de entrada y salida, el objetivo principal del concurso, un análisis exploratorio básico, los procedimientos de codificación de valores faltantes y variables categóricas, así como las estrategias de
ingeniería de características —si aplican—. Finalmente, proporcione todos los detalles necesarios para comprender, interpretar y analizar adecuadamente el conjunto de datos.

Link del kaggle donde se hizo la exploracion de datos
[Punto 4.1](https://www.kaggle.com/code/julopezrio/parcial1-p4-1/)

## Descripción
El dataset de la NFL 2026 es un conjunto de datos de las jugadas realizadas a lo largo de diferentes juegos en la temporada, su función principal es permitir la predicción de la posición de un jugador a lo largo de un frame de análisis tomando como base diferentes características como la velocidad, orientación, dirección de juego, posición actual del jugador, rol, entre otras. La salida del modelo debe entregar las variables target (posición X y Y de los jugadores) al momento de lanzar un balón.
Para esto, la tarea fundamental es crear un modelo de predicción de ubicación espacial y temporal al momento de analizar un frame de juego.

#Punto #4.2: Modelo
Luego, utilizando un esquema de validacion hold-out con una partición del 60% para entrenamiento, 20 % para validacion y 20% para evaluación del rendimiento, compare los regresores mencionados en el punto 2 —considerando sus posibles implementaciones en RAPIDS, segun lo indicado en el punto 3—, empleando el conjunto de datos de entrenamiento disponible en el dataset.
Realice las consideraciones necesarias para la implementacion y sintonización de cada modelo mediante optimización bayesiana, con el fin de determinar los hiperparámetros más relevantes de cada algoritmo. Justifique la selección de los hiperparámetros a optimizar, así como la rejilla o los rangos de valores definidos para cada modelo, de acuerdo con las características estudiadas
y el score a minimizar. Finalmente, presente los rendimientos promedio obtenidos sobre el conjunto de evaluacion, junto con su respectiva desviación estándar, empleando las siguientes métricas de desempeño: MAE, MSE, R2 (consultar) y MAPE (consultar).

In [ ]:
# Librerias
!pip install optuna
import optuna
from optuna.samplers import GPSampler
from functools import partial
from cuml.linear_model import LinearRegression, Lasso, ElasticNet
from cuml.metrics import mean_squared_error, r2_score, mean_absolute_error
from cuml.ensemble import RandomForestRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import SGDRegressor, BayesianRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.gaussian_process.kernels import RBF, Matern, RationalQuadratic, DotProduct, WhiteKernel
from xgboost import XGBRegressor
from cuml.model_selection import train_test_split
from cuml.preprocessing import StandardScaler

clear_output()

def error_calculation(pred, real):
  """
  Esta función calcula los diferentes tipos de errores
  sobre la predicción de los modelos en base a los datos
  de testeo.
  """
  if hasattr(pred, "to_numpy"):
    pred = pred.to_numpy()
  if hasattr(real, "to_numpy"):
    real = real.to_numpy()
  mse = mean_squared_error(real, pred)
  mae = mean_absolute_error(real, pred)
  r2 = r2_score(real, pred)
  mape = mean_absolute_percentage_error(real, pred)
  errores = [mse, mae, r2, mape]
  return errores

In [ ]:
# Partición de los datos del dataset arreglado y filtrado
X_train_val, X_test, y_train_val, y_test = train_test_split(
  train_full, targets, test_size=0.2, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(
  X_train_val, y_train_val, test_size=0.25, random_state=42)

# Crear el escalador
scaler = StandardScaler()

# Hacer FIT solo en los datos de entrada
X_train_scaled = scaler.fit_transform(X_train)

# Transformar validación y test en base al escalador
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Tamaño de los sets
print("Train:", X_train.shape)
print("Validation:", X_val.shape)
print("Test:", X_test.shape)

Train: (336256, 18)
Validation: (112085, 18)
Test: (112085, 18)


In [ ]:
# Organización del conjunto de datos para su uso en Optuna
X_train_scaled, y_train = X_train_scaled.iloc[:, :-1], X_train_scaled.iloc[:, -1]
X_val_scaled, y_val = X_val_scaled.iloc[:, :-1], X_val_scaled.iloc[:, -1]
X_test_scaled, y_test = X_test_scaled.iloc[:, :-1], X_test_scaled.iloc[:, -1]

# --- Modelos ---
GPU_regressors = {
  'LinearRegression': LinearRegression,
  'Lasso': Lasso,
  'ElasticNet': ElasticNet,
  'RandomForestRegressor': RandomForestRegressor,
  'XGBRegressor': XGBRegressor
}

# --- Espacio de búsqueda, información de cada regresor para Optuna ---
search_spaces_GPU = {
  'LinearRegression': lambda trial: {
    "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
  },
  'Lasso': lambda trial: {
    "alpha": trial.suggest_float("alpha", 1e-30, 10),
    "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
  },
  'ElasticNet': lambda trial: {
    "alpha": trial.suggest_float("alpha", 1e-30, 10),
    "l1_ratio": trial.suggest_float("l1_ratio", 0.0, 1.0),
    "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
  },
  'RandomForestRegressor': lambda trial: {
    "n_estimators": trial.suggest_int("n_estimators", 10, 100),
    "max_depth": trial.suggest_int("max_depth", 3, 15),
    "max_features": trial.suggest_float("max_features", 0, 1),
  },
  'XGBRegressor': lambda trial: {
    "n_estimators": trial.suggest_int("n_estimators", 10, 100),
    "max_depth": trial.suggest_int("max_depth", 3, 8),
    "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3),
    "subsample": trial.suggest_float("subsample", 0.6, 0.9),
    "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 0.9),
    "reg_lambda": trial.suggest_float("reg_lambda", 1e-3, 10.0),
    "tree_method": "gpu_hist",
    "predictor": "gpu_predictor",
    "max_bin": trial.suggest_int("max_bin", 64, 256),
    "verbosity": 0
  }
}

# --- Conteo de intentos ---
trial_counts_GPU = {
  'LinearRegression': 10,
  'Lasso': 30,
  'ElasticNet': 30,
  'RandomForestRegressor': 20,
  'XGBRegressor': 20
}

# --- Optimización ---
def run_optimization_GPU(model_name, ModelClass, search_space, n_trials):
    """
    Ejecuta la optimización bayesiana para el modelo especificado.
    """
    def objective(trial):
        params = search_space(trial)
        model = ModelClass(**params)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        errores = error_calculation(y_val, y_pred)
        error = errores[0]
        return error

    study = optuna.create_study(direction='minimize', sampler=GPSampler())
    study.optimize(objective, n_trials=n_trials)

    print(f"\n✅ {model_name} optimized | Best params: {study.best_params}, error={study.best_value:.4f}")
    return {
        "study": study,
        "best_params": study.best_params,
        "best_error": study.best_value,
    }

# --- Main ---
def optimizacion_GPU():
  """
  Optimiza cada modelo de GPU y devuelve los mejores resultados.
  """
  best_results = {}
  for name, ModelClass in GPU_regressors.items():
    best_results[name] = run_optimization_GPU(
      name,
      ModelClass,
      search_spaces_GPU[name],
      trial_counts_GPU[name]
    )
  return best_results

# --- Evaluar el modelo con TEST ---
def evaluate_best_models(best_results):
    """Retrains best models on full training set and evaluates on test data."""
    test_results = {}
    for name, info in best_results.items():
        model = GPU_regressors[name](**info["best_params"])
        model.fit(X_train_scaled, y_train)
        y_pred_test = model.predict(X_test_scaled)
        errores = error_calculation(y_test, y_pred_test)
        test_results[name] = {
            "params": info["best_params"],
            "validation_error": info["best_error"],
            "MSE": errores[0],
            "MAE": errores[1],
            "R2": errores[2],
            "MAPE": errores[3]
        }
        print(f"\n🔎 {name} → Test error: {errores[0]:.4f}")
    return test_results

In [ ]:
# Step 1: Optimize
best_results = optimizacion_GPU()

# Step 2: Evaluate best models on test data
test_summary1 = evaluate_best_models(best_results)

[I 2025-10-15 01:50:54,257] A new study created in memory with name: no-name-7211de1d-da88-49b3-9646-4420b171bae0
[I 2025-10-15 01:50:57,457] Trial 0 finished with value: 0.01056117504462794 and parameters: {'fit_intercept': True}. Best is trial 0 with value: 0.01056117504462794.
[I 2025-10-15 01:50:58,088] Trial 1 finished with value: 0.01056117504462789 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 0.01056117504462789.
[I 2025-10-15 01:50:58,562] Trial 2 finished with value: 0.01056117504462794 and parameters: {'fit_intercept': True}. Best is trial 1 with value: 0.01056117504462789.
[I 2025-10-15 01:50:59,038] Trial 3 finished with value: 0.01056117504462789 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 0.01056117504462789.
[I 2025-10-15 01:50:59,510] Trial 4 finished with value: 0.01056117504462789 and parameters: {'fit_intercept': False}. Best is trial 1 with value: 0.01056117504462789.
[I 2025-10-15 01:50:59,996] Trial 5 finished wit


✅ LinearRegression optimized | Best params: {'fit_intercept': False}, error=0.0106


[I 2025-10-15 01:51:02,242] Trial 3 finished with value: 1.0050609704817863 and parameters: {'alpha': 8.249919349026445, 'fit_intercept': False}. Best is trial 2 with value: 0.5586658410227278.
[I 2025-10-15 01:51:02,308] Trial 4 finished with value: 0.6487232763075215 and parameters: {'alpha': 0.7912694678151777, 'fit_intercept': True}. Best is trial 2 with value: 0.5586658410227278.
[I 2025-10-15 01:51:02,363] Trial 5 finished with value: 1.0050609704817863 and parameters: {'alpha': 7.265220987479873, 'fit_intercept': False}. Best is trial 2 with value: 0.5586658410227278.
[I 2025-10-15 01:51:02,420] Trial 6 finished with value: 1.0050609704817863 and parameters: {'alpha': 6.856416783419146, 'fit_intercept': True}. Best is trial 2 with value: 0.5586658410227278.
[I 2025-10-15 01:51:02,468] Trial 7 finished with value: 1.0050609704817863 and parameters: {'alpha': 9.624733082363761, 'fit_intercept': True}. Best is trial 2 with value: 0.5586658410227278.
[I 2025-10-15 01:51:02,541] Tria


✅ Lasso optimized | Best params: {'alpha': 1e-30, 'fit_intercept': False}, error=0.0106


[I 2025-10-15 01:51:16,070] Trial 6 finished with value: 0.4227074920111168 and parameters: {'alpha': 2.5304268383797646, 'l1_ratio': 0.025833557391694684, 'fit_intercept': True}. Best is trial 5 with value: 0.24133700284958637.
[I 2025-10-15 01:51:16,091] Trial 7 finished with value: 1.0050609704817863 and parameters: {'alpha': 1.246998149167824, 'l1_ratio': 0.92847306118329, 'fit_intercept': False}. Best is trial 5 with value: 0.24133700284958637.
[I 2025-10-15 01:51:16,111] Trial 8 finished with value: 1.0050609704817863 and parameters: {'alpha': 5.993593133668296, 'l1_ratio': 0.6089725781493727, 'fit_intercept': False}. Best is trial 5 with value: 0.24133700284958637.
[I 2025-10-15 01:51:16,135] Trial 9 finished with value: 1.0050609704817863 and parameters: {'alpha': 8.327142396175057, 'l1_ratio': 0.9751048423853275, 'fit_intercept': True}. Best is trial 5 with value: 0.24133700284958637.
[I 2025-10-15 01:51:16,397] Trial 10 finished with value: 0.010559646453440557 and parameters


✅ ElasticNet optimized | Best params: {'alpha': 1e-30, 'l1_ratio': 0.7550916423368342, 'fit_intercept': False}, error=0.0106


[I 2025-10-15 01:51:25,792] Trial 0 finished with value: 0.0007381520153584586 and parameters: {'n_estimators': 53, 'max_depth': 14, 'max_features': 0.4232982677329047}. Best is trial 0 with value: 0.0007381520153584586.
[I 2025-10-15 01:51:26,864] Trial 1 finished with value: 0.01731882457738181 and parameters: {'n_estimators': 48, 'max_depth': 4, 'max_features': 0.7158849896091383}. Best is trial 0 with value: 0.0007381520153584586.
[I 2025-10-15 01:51:28,889] Trial 2 finished with value: 0.008617245937078046 and parameters: {'n_estimators': 56, 'max_depth': 15, 'max_features': 0.191925386121793}. Best is trial 0 with value: 0.0007381520153584586.
[I 2025-10-15 01:51:32,087] Trial 3 finished with value: 0.0005732641382991015 and parameters: {'n_estimators': 86, 'max_depth': 11, 'max_features': 0.8308294357492965}. Best is trial 3 with value: 0.0005732641382991015.
[I 2025-10-15 01:51:33,531] Trial 4 finished with value: 0.5235695941274902 and parameters: {'n_estimators': 76, 'max_dep


✅ RandomForestRegressor optimized | Best params: {'n_estimators': 100, 'max_depth': 15, 'max_features': 0.8577001737723099}, error=0.0005


[I 2025-10-15 01:52:17,512] Trial 0 finished with value: 0.004081071569933154 and parameters: {'n_estimators': 63, 'max_depth': 8, 'learning_rate': 0.29903258257498216, 'subsample': 0.8124234365861058, 'colsample_bytree': 0.7928954275194406, 'reg_lambda': 8.76751985223971, 'max_bin': 82}. Best is trial 0 with value: 0.004081071569933154.
[I 2025-10-15 01:52:17,729] Trial 1 finished with value: 0.0034029850599482446 and parameters: {'n_estimators': 80, 'max_depth': 3, 'learning_rate': 0.15909608420347973, 'subsample': 0.6473593803496623, 'colsample_bytree': 0.8283981610606008, 'reg_lambda': 1.5495296042977575, 'max_bin': 88}. Best is trial 1 with value: 0.0034029850599482446.
[I 2025-10-15 01:52:18,052] Trial 2 finished with value: 0.0021101994596628266 and parameters: {'n_estimators': 94, 'max_depth': 5, 'learning_rate': 0.24545135043100758, 'subsample': 0.7920699871711522, 'colsample_bytree': 0.8025244419266574, 'reg_lambda': 2.140080348401779, 'max_bin': 102}. Best is trial 2 with va


✅ XGBRegressor optimized | Best params: {'n_estimators': 83, 'max_depth': 8, 'learning_rate': 0.14968663701483587, 'subsample': 0.634943863233365, 'colsample_bytree': 0.8849697827285405, 'reg_lambda': 8.761637380191186, 'max_bin': 125}, error=0.0009

🔎 LinearRegression → Test error: 0.0105

🔎 Lasso → Test error: 0.0105

🔎 ElasticNet → Test error: 0.0105

🔎 RandomForestRegressor → Test error: 0.0005

🔎 XGBRegressor → Test error: 0.0006


In [ ]:
# Conversión del dataset para trabajar con CPU
X_train_scaled, y_train = X_train_scaled.to_pandas(), X_train_scaled.to_pandas()
X_val_scaled, y_val = X_val_scaled.to_pandas(), X_val_scaled.to_pandas()
X_test_scaled, y_test = X_test_scaled.to_pandas(), X_test_scaled.to_pandas()

# Organización del conjunto de datos para su uso en Optuna
X_train_scaled, y_train = X_train_scaled.iloc[:, :-1], X_train_scaled.iloc[:, -1]
X_val_scaled, y_val = X_val_scaled.iloc[:, :-1], X_val_scaled.iloc[:, -1]
X_test_scaled, y_test = X_test_scaled.iloc[:, :-1], X_test_scaled.iloc[:, -1]

# --- Modelos ---
CPU_regressors = {
    'Bayesian Ridge': BayesianRidge,
    'SGD Regressor': SGDRegressor,
}

# --- Espacio de búsqueda, información de cada regresor para Optuna ---
search_spaces_CPU = {
  'Bayesian Ridge': lambda trial: {
    "alpha_1": trial.suggest_float("alpha_1", 1e-7, 1e-3),
    "alpha_2": trial.suggest_float("alpha_2", 1e-7, 1e-3),
    "lambda_1": trial.suggest_float("lambda_1", 1e-7, 1e-3),
    "lambda_2": trial.suggest_float("lambda_2", 1e-7, 1e-3),
    "fit_intercept": trial.suggest_categorical("fit_intercept", [True, False]),
  },
  'SGD Regressor': lambda trial: {
    "loss": trial.suggest_categorical("loss", ["squared_error", "huber", "epsilon_insensitive"]),
    "penalty": trial.suggest_categorical("penalty", ["l2", "l1", "elasticnet"]),
    "alpha": trial.suggest_float("alpha", 1e-5, 1e-1),
    "l1_ratio": trial.suggest_float("l1_ratio", 0.0, 1.0),
    "max_iter": 1000,
    "tol": 1e-3,
    "learning_rate": trial.suggest_categorical("learning_rate", ["constant", "optimal", "invscaling", "adaptive"]),
    "eta0": trial.suggest_float("eta0", 1e-4, 1e-1)
  }
}

# --- Conteo de intentos ---
trial_counts_CPU = {
  'Bayesian Ridge': 20,
  'SGD Regressor': 20
}

# --- Optimización ---
def run_optimization_CPU(model_name, ModelClass, search_space, n_trials):
    """
    Ejecuta la optimización bayesiana para el modelo especificado.
    """
    def objective(trial):
        params = search_space(trial)
        model = ModelClass(**params)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        errores = error_calculation(y_val, y_pred)
        error = errores[0]
        return error

    study = optuna.create_study(direction='minimize', sampler=GPSampler())
    study.optimize(objective, n_trials=n_trials)

    print(f"\n✅ {model_name} optimized | Best params: {study.best_params}, error={study.best_value:.4f}")
    return {
        "study": study,
        "best_params": study.best_params,
        "best_error": study.best_value
    }

# --- Main ---
def optimizacion_CPU():
  """
  Optimiza cada modelo de CPU y devuelve los mejores resultados.
  """

  best_results = {}
  for name, ModelClass in CPU_regressors.items():
    best_results[name] = run_optimization_CPU(
      name,
      ModelClass,
      search_spaces_CPU[name],
      trial_counts_CPU[name]
    )
  return best_results

# --- Evaluar el modelo con TEST ---
def evaluate_best_models(best_results):
    """Retrains best models on full training set and evaluates on test data."""
    test_results = {}
    for name, info in best_results.items():
        model = CPU_regressors[name](**info["best_params"])
        model.fit(X_train_scaled, y_train)
        y_pred_test = model.predict(X_test_scaled)
        errores = error_calculation(y_test, y_pred_test)
        test_results[name] = {
            "params": info["best_params"],
            "validation_error": info["best_error"],
            "MSE": errores[0],
            "MAE": errores[1],
            "R2": errores[2],
            "MAPE": errores[3]
        }
        print(f"\n🔎 {name} → Test error: {errores[0]:.4f}")
    return test_results

In [ ]:
# Step 1: Optimize
best_results = optimizacion_CPU()

# Step 2: Evaluate best models on test data
test_summary2 = evaluate_best_models(best_results)

[I 2025-10-15 01:52:48,618] A new study created in memory with name: no-name-a371703b-2ce8-4971-8faf-8f8d9972bd44
[I 2025-10-15 01:52:48,904] Trial 0 finished with value: 0.8034496220155124 and parameters: {'alpha_1': 0.00029351974690257495, 'alpha_2': 0.0006039575205431093, 'lambda_1': 0.00047835861315536753, 'lambda_2': 4.845876296645411e-06, 'fit_intercept': False}. Best is trial 0 with value: 0.8034496220155124.
[I 2025-10-15 01:52:49,219] Trial 1 finished with value: 0.8034496226191089 and parameters: {'alpha_1': 0.0004136134337482957, 'alpha_2': 0.0003738719473087968, 'lambda_1': 0.0007899827798794424, 'lambda_2': 6.19865751509749e-05, 'fit_intercept': False}. Best is trial 0 with value: 0.8034496220155124.
[I 2025-10-15 01:52:49,524] Trial 2 finished with value: 0.8034496229882505 and parameters: {'alpha_1': 0.0001879797517657989, 'alpha_2': 7.744652249111829e-05, 'lambda_1': 0.00010459893930948939, 'lambda_2': 8.822054077175467e-05, 'fit_intercept': False}. Best is trial 0 with


✅ Bayesian Ridge optimized | Best params: {'alpha_1': 1e-07, 'alpha_2': 0.001, 'lambda_1': 0.001, 'lambda_2': 1e-07, 'fit_intercept': False}, error=0.8034


[I 2025-10-15 01:53:02,641] Trial 0 finished with value: 0.8415168752181525 and parameters: {'loss': 'epsilon_insensitive', 'penalty': 'elasticnet', 'alpha': 0.06078438447298159, 'l1_ratio': 0.8082795816618378, 'learning_rate': 'adaptive', 'eta0': 0.08046480742317914}. Best is trial 0 with value: 0.8415168752181525.
[I 2025-10-15 01:53:03,290] Trial 1 finished with value: 1.0364542936530978 and parameters: {'loss': 'huber', 'penalty': 'l1', 'alpha': 0.07677502874789767, 'l1_ratio': 0.2408123422983185, 'learning_rate': 'optimal', 'eta0': 0.09851349312510296}. Best is trial 0 with value: 0.8415168752181525.
[I 2025-10-15 01:54:15,089] Trial 2 finished with value: 5085100707.809462 and parameters: {'loss': 'squared_error', 'penalty': 'elasticnet', 'alpha': 0.08591204894592964, 'l1_ratio': 0.5050286388283649, 'learning_rate': 'optimal', 'eta0': 0.06683935345385163}. Best is trial 0 with value: 0.8415168752181525.
[I 2025-10-15 01:54:19,483] Trial 3 finished with value: 0.8102529800821539 a


✅ SGD Regressor optimized | Best params: {'loss': 'squared_error', 'penalty': 'elasticnet', 'alpha': 0.08743463680812863, 'l1_ratio': 0.3597496988223887, 'learning_rate': 'invscaling', 'eta0': 0.0941621226488407}, error=0.8095

🔎 Bayesian Ridge → Test error: 0.8029

🔎 SGD Regressor → Test error: 0.8077


In [ ]:
# Recorte de datos para otros regresores
X_train_scaled, y_train = X_train_scaled[0:5000], X_train_scaled[0:5000]
X_val_scaled, y_val = X_val_scaled[0:5000], X_val_scaled[0:5000]
X_test_scaled, y_test = X_test_scaled[0:5000], X_test_scaled[0:5000]

# Organización del conjunto de datos para su uso en Optuna
X_train_scaled, y_train = X_train_scaled.iloc[:, :-1], X_train_scaled.iloc[:, -1]
X_val_scaled, y_val = X_val_scaled.iloc[:, :-1], X_val_scaled.iloc[:, -1]
X_test_scaled, y_test = X_test_scaled.iloc[:, :-1], X_test_scaled.iloc[:, -1]

# --- Modelos ---
Limited_regressors = {
    'SVR': SVR,
    'KernelRidge': KernelRidge
}

# --- Espacio de búsqueda, información de cada regresor para Optuna ---
search_spaces_limited = {
  'SVR': lambda trial: {
      "C": trial.suggest_float("C", 1e-3, 100),
      "epsilon": trial.suggest_float("epsilon", 1e-4, 1.0),
      "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf"]),
  },
  'KernelRidge': lambda trial: {
    "alpha": trial.suggest_float("alpha", 1e-3, 10.0),
    "kernel": trial.suggest_categorical("kernel", ["linear", "poly", "rbf", "sigmoid"]),
    "gamma": trial.suggest_float("gamma", 1e-4, 1.0),
    "degree": trial.suggest_int("degree", 2, 4),
    "coef0": trial.suggest_float("coef0", 0.0, 1.0),
  }
}

# --- Conteo de intentos ---
trial_counts_limited = {
  'SVR': 20,
  'KernelRidge': 20
}

# --- Optimización ---
def run_optimization_limited(model_name, ModelClass, search_space, n_trials):
    """
    Ejecuta la optimización bayesiana para el modelo especificado.
    """
    def objective(trial):
        params = search_space(trial)
        model = ModelClass(**params)
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_val_scaled)
        errores = error_calculation(y_val, y_pred)
        error = errores[0]
        return error

    study = optuna.create_study(direction='minimize', sampler=GPSampler())
    study.optimize(objective, n_trials=n_trials)

    print(f"\n✅ {model_name} optimized | Best params: {study.best_params}, error={study.best_value:.4f}")
    return {
        "study": study,
        "best_params": study.best_params,
        "best_error": study.best_value
    }

# --- Main ---
def optimizacion_limited():
  """
  Optimiza cada modelo de CPU y devuelve los mejores resultados.
  """

  best_results = {}
  for name, ModelClass in Limited_regressors.items():
    best_results[name] = run_optimization_CPU(
      name,
      ModelClass,
      search_spaces_limited[name],
      trial_counts_limited[name]
    )
  return best_results

# --- Evaluar el modelo con TEST ---
def evaluate_best_models(best_results):
    """Retrains best models on full training set and evaluates on test data."""
    test_results = {}
    for name, info in best_results.items():
        model = Limited_regressors[name](**info["best_params"])
        model.fit(X_train_scaled, y_train)
        y_pred_test = model.predict(X_test_scaled)
        errores = error_calculation(y_test, y_pred_test)
        test_results[name] = {
            "params": info["best_params"],
            "validation_error": info["best_error"],
            "MSE": errores[0],
            "MAE": errores[1],
            "R2": errores[2],
            "MAPE": errores[3]
        }
        print(f"\n🔎 {name} → Test error: {errores[0]:.4f}")
    return test_results

In [ ]:
# Step 1: Optimize
best_results = optimizacion_limited()

# Step 2: Evaluate best models on test data
test_summary3 = evaluate_best_models(best_results)

[I 2025-10-15 01:56:04,031] A new study created in memory with name: no-name-9fe510b6-8ada-4d93-994d-bc2719a8afbf
[I 2025-10-15 01:56:21,777] Trial 0 finished with value: 0.5153826165774967 and parameters: {'C': 26.279676351079143, 'epsilon': 0.3059332937620042, 'kernel': 'poly'}. Best is trial 0 with value: 0.5153826165774967.
[I 2025-10-15 01:56:22,755] Trial 1 finished with value: 0.5114850709600285 and parameters: {'C': 2.188867993349816, 'epsilon': 0.9503349403174572, 'kernel': 'linear'}. Best is trial 1 with value: 0.5114850709600285.
[I 2025-10-15 01:57:00,473] Trial 2 finished with value: 0.4938264099672456 and parameters: {'C': 57.77062691855204, 'epsilon': 0.4796287390437881, 'kernel': 'linear'}. Best is trial 2 with value: 0.4938264099672456.
[I 2025-10-15 01:57:08,617] Trial 3 finished with value: 0.5487790171669592 and parameters: {'C': 33.62701222189372, 'epsilon': 0.20193705935328635, 'kernel': 'rbf'}. Best is trial 2 with value: 0.4938264099672456.
[I 2025-10-15 01:57:1


✅ SVR optimized | Best params: {'C': 13.911079360236261, 'epsilon': 0.4890854517447266, 'kernel': 'rbf'}, error=0.4876


[I 2025-10-15 02:03:38,952] Trial 0 finished with value: 0.464591569734334 and parameters: {'alpha': 9.347856999035123, 'kernel': 'poly', 'gamma': 0.7853105431141525, 'degree': 2, 'coef0': 0.3100353096661608}. Best is trial 0 with value: 0.464591569734334.
[I 2025-10-15 02:03:40,687] Trial 1 finished with value: 0.49241960646596433 and parameters: {'alpha': 1.4992740433846916, 'kernel': 'linear', 'gamma': 0.44638758539394147, 'degree': 2, 'coef0': 0.9896017654988448}. Best is trial 0 with value: 0.464591569734334.
[I 2025-10-15 02:03:42,618] Trial 2 finished with value: 0.6357443950378143 and parameters: {'alpha': 2.194598134121174, 'kernel': 'rbf', 'gamma': 0.5777436304817978, 'degree': 2, 'coef0': 0.9478855894882369}. Best is trial 0 with value: 0.464591569734334.
[I 2025-10-15 02:03:44,538] Trial 3 finished with value: 0.7771528468470904 and parameters: {'alpha': 3.4878839116919327, 'kernel': 'rbf', 'gamma': 0.8388178361545897, 'degree': 4, 'coef0': 0.8787799551892076}. Best is tria


✅ KernelRidge optimized | Best params: {'alpha': 8.053080674261649, 'kernel': 'poly', 'gamma': 0.6883619869531428, 'degree': 2, 'coef0': 0.631345310096817}, error=0.4645

🔎 SVR → Test error: 0.4971

🔎 KernelRidge → Test error: 0.4689


In [ ]:
import pandas as pd

def build_results_table(test_summary):
    """
    Builds a clean comparison table from the test_summary dict.
    """
    rows = []
    for model, res in test_summary.items():
        rows.append({
            "Model": model,
            "MSE": round(res["validation_error"], 4),
            "Test Error": round(res["MSE"], 4),
            "MAE": round(res["MAE"], 4),
            "R2": round(res["R2"], 4),
            "MAPE": round(res["MAPE"], 4),
            "Best Params": res["params"]
        })
    df_results = pd.DataFrame(rows)
    df_results = df_results.sort_values(by="Test Error", ascending=True).reset_index(drop=True)
    return df_results

results_table_gpu = build_results_table(test_summary1)
results_table_cpu = build_results_table(test_summary2)
results_table_limited = build_results_table(test_summary3)

results_table = pd.concat([results_table_gpu, results_table_cpu], ignore_index=True)
results_table = pd.concat([results_table_limited, results_table], ignore_index=True)

print("\n=== Model Comparison ===")
display(results_table)


=== Model Comparison ===


,Model,MSE,Test Error,MAE,R2,MAPE,Best Params
0,KernelRidge,0.4645,0.4689,0.5476,0.1529,3.4493,"{'alpha': 8.053080674261649, 'kernel': 'poly',..."
1,SVR,0.4876,0.4971,0.5531,0.1887,3.3498,"{'C': 13.911079360236261, 'epsilon': 0.4890854..."
2,RandomForestRegressor,0.0005,0.0005,0.0096,0.9995,0.0682,"{'n_estimators': 100, 'max_depth': 15, 'max_fe..."
3,XGBRegressor,0.0009,0.0006,0.0121,0.9994,0.0863,"{'n_estimators': 83, 'max_depth': 8, 'learning..."
4,Lasso,0.0106,0.0105,0.0635,0.9893,0.1540,"{'alpha': 1e-30, 'fit_intercept': False}"
5,LinearRegression,0.0106,0.0105,0.0635,0.9893,0.1676,{'fit_intercept': False}
6,ElasticNet,0.0106,0.0105,0.0635,0.9893,0.1540,"{'alpha': 1e-30, 'l1_ratio': 0.755091642336834..."
7,Bayesian Ridge,0.8034,0.8029,0.6981,-3.1656,19.9512,"{'alpha_1': 1e-07, 'alpha_2': 0.001, 'lambda_1..."
8,SGD Regressor,0.8095,0.8077,0.7069,-4.0702,65.4197,"{'loss': 'squared_error', 'penalty': 'elasticn..."
